In [21]:
import pandas as pd
import sqlite3
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

In [22]:
movies = pd.read_csv("https://raw.githubusercontent.com/4GeeksAcademy/k-nearest-neighbors-project-tutorial/main/tmdb_5000_movies.csv")
credits = pd.read_csv("https://raw.githubusercontent.com/4GeeksAcademy/k-nearest-neighbors-project-tutorial/main/tmdb_5000_credits.csv")

movies.sample(5)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
4237,0,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 27, ""nam...",http://www.hatchetmovie.com/,11908,"[{""id"": 2411, ""name"": ""new orleans""}, {""id"": 3...",en,Hatchet,When a group of tourists on a New Orleans haun...,12.677016,"[{""name"": ""ArieScope Pictures"", ""id"": 3960}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2006-04-27,0,83.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,Stay out of the swamp.,Hatchet,5.7,159
873,46000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",NaN,479,"[{""id"": 417, ""name"": ""corruption""}, {""id"": 577...",en,Shaft,New York police detective John Shaft arrests W...,19.643365,"[{""name"": ""Paramount Pictures"", ""id"": 4}, {""na...","[{""iso_3166_1"": ""DE"", ""name"": ""Germany""}, {""is...",2000-06-15,107196498,99.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,"Still the man, any questions?",Shaft,5.5,308
3924,3800000,"[{""id"": 35, ""name"": ""Comedy""}]",NaN,10557,"[{""id"": 187, ""name"": ""islam""}, {""id"": 458, ""na...",en,East Is East,In 1971 Salford fish-and-chip shop owner Georg...,3.667372,"[{""name"": ""BBC"", ""id"": 5996}, {""name"": ""Assass...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""}]",1999-05-14,28200000,97.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,"A comedy of families, a chip shop... and a ver...",East Is East,6.5,59
2173,20000000,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...",NaN,13159,"[{""id"": 5987, ""name"": ""grandmother granddaught...",en,Georgia Rule,"Georgia Rule follows a rebellious, uncontrolla...",7.878177,"[{""name"": ""Universal Pictures"", ""id"": 33}, {""n...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-11,0,113.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,"In this family, attitude doesn't skip a genera...",Georgia Rule,5.6,130
4212,0,"[{""id"": 18, ""name"": ""Drama""}, {""id"": 9648, ""na...",NaN,25428,[],en,I Love Your Work,"A fictional movie star, Gray Evans, goes throu...",3.313339,[],[],2003-10-05,0,111.0,"[{""iso_639_1"": ""de"", ""name"": ""Deutsch""}]",Released,NaN,I Love Your Work,6.5,2


In [23]:
conn = sqlite3.connect("movies_database.db")

movies.to_sql("movies_table", conn, if_exists = "replace", index = False)
credits.to_sql("credits_table", conn, if_exists = "replace", index = False)

4803

In [24]:
# table merge

query = """
    SELECT *
    FROM movies_table
    INNER JOIN credits_table
    ON movies_table.title = credits_table.title;
"""

total_data = pd.read_sql_query(query, conn)
conn.close()

total_data = total_data.loc[:, ~total_data.columns.duplicated()]
total_data.sample(5)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
3925,3000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 27, ""nam...",http://phantasm.com,15158,"[{""id"": 7939, ""name"": ""portal""}, {""id"": 12434,...",en,Phantasm II,"Mike, after his release from a psychiatric hos...",5.003615,"[{""name"": ""Universal Pictures"", ""id"": 33}, {""n...",...,97.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The ball is back!,Phantasm II,6.3,94,15158,"[{""cast_id"": 4, ""character"": ""Mike"", ""credit_i...","[{""credit_id"": ""535f7e3f0e0a264fde007a18"", ""de..."
2343,18000000,"[{""id"": 10752, ""name"": ""War""}, {""id"": 35, ""nam...",None,10364,"[{""id"": 1956, ""name"": ""world war ii""}, {""id"": ...",en,Catch-22,A bombardier in World War II tries desperately...,4.504377,"[{""name"": ""Paramount Pictures"", ""id"": 4}, {""na...",...,121.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,None,Catch-22,6.7,72,10364,"[{""cast_id"": 1, ""character"": ""Captain John Yos...","[{""credit_id"": ""56d9117e9251412fe8005083"", ""de..."
593,70000000,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...",http://www.thedilemmamovie.com,44564,"[{""id"": 596, ""name"": ""adultery""}, {""id"": 1326,...",en,The Dilemma,Longtime friends Ronny and Nick are partners i...,14.780634,"[{""name"": ""Imagine Entertainment"", ""id"": 23}, ...",...,111.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,Two best friends. Nothing could come between t...,The Dilemma,5.2,304,44564,"[{""cast_id"": 4, ""character"": ""Nick Brannen"", ""...","[{""credit_id"": ""58d402a8c3a368121b0159b8"", ""de..."
910,50000000,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10749, ""...",None,9919,"[{""id"": 242, ""name"": ""new york""}, {""id"": 622, ...",en,How to Lose a Guy in 10 Days,"An advice columnist, Andie Anderson (Kate Huds...",23.854701,"[{""name"": ""Paramount Pictures"", ""id"": 4}]",...,116.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,One of them is lying. So is the other.,How to Lose a Guy in 10 Days,6.3,850,9919,"[{""cast_id"": 12, ""character"": ""Andie Anderson""...","[{""credit_id"": ""52fe4549c3a36847f80c59c1"", ""de..."
2469,16000000,"[{""id"": 80, ""name"": ""Crime""}, {""id"": 18, ""name...",None,16161,"[{""id"": 641, ""name"": ""single parent""}, {""id"": ...",en,Baby Boy,"The story of Jody, a misguided, 20-year-old Af...",7.854989,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",...,130.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,Just a Baby Boy.,Baby Boy,7.4,55,16161,"[{""cast_id"": 1, ""character"": ""Joseph Summers"",...","[{""credit_id"": ""52fe46ae9251416c7507ffa1"", ""de..."


In [25]:
# data transform

def load_json_safe(json_str, default_value = None):
    try:
        return json.loads(json_str)
    except (TypeError, json.JSONDecodeError):
        return default_value
    
total_data["genres"] = total_data["genres"].apply(lambda x: [item["name"] for item in json.loads(x)] if pd.notna(x) else None)
total_data["keywords"] = total_data["keywords"].apply(lambda x: [item["name"] for item in json.loads(x)] if pd.notna(x) else None)

total_data["cast"] = total_data["cast"].apply(lambda x: [item["name"] for item in json.loads(x)][:3] if pd.notna(x) else None)

total_data["crew"] = total_data["crew"].apply(lambda x: " ".join([crew_member['name'] for crew_member in load_json_safe(x) if crew_member['job'] == 'Director']))

total_data["overview"] = total_data["overview"].apply(lambda x: [x])

total_data.sample(5)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
816,55000000,"[Adventure, Comedy, Family, Romance]",None,10603,"[africa, san francisco, gorilla, lion, feral c...",en,George of the Jungle,[Baby George got into a plane crash in a jungl...,22.573237,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,92.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Watch out!,George of the Jungle,5.4,496,10603,"[Brendan Fraser, Leslie Mann, Thomas Haden Chu...",Sam Weisman
2756,6000000,"[Comedy, Romance]",None,25462,[],en,Two Can Play That Game,[Vivica A. Fox sizzles as a woman scorned who ...,2.272130,"[{""name"": ""Screen Gems"", ""id"": 3287}]",...,90.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,None,Two Can Play That Game,6.6,17,25462,"[Vivica A. Fox, Morris Chestnut, Anthony Ander...",Mark Brown
663,65000000,[Drama],None,11170,"[american football, 1970s, trainer, college, s...",en,We Are Marshall,[When a plane crash claims the lives of member...,8.629665,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...",...,124.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,From the ashes we rose.,We Are Marshall,6.7,186,11170,"[Matthew McConaughey, Matthew Fox, Anthony Mac...",McG
1893,25000000,"[Drama, Romance, War]",None,22971,"[based on novel, army, love, u.s. soldier]",en,Dear John,[Sergeant John Tyree is home on a 2 week leave...,19.402816,"[{""name"": ""Screen Gems"", ""id"": 3287}]",...,115.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,Love brought them together. Will fate tear the...,Dear John,6.6,1262,22971,"[Amanda Seyfried, Channing Tatum, Henry Thomas]",Lasse Hallström
4723,125000,"[Drama, Comedy]",http://www.yoursisterssister-themovie.com/,84355,"[sister sister relationship, secret, romance, ...",en,Your Sister's Sister,[Iris invites her friend Jack to stay at her f...,12.833247,"[{""name"": ""Ada Films"", ""id"": 1424}]",...,90.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,A comedy about doing the right thing with the ...,Your Sister's Sister,6.5,137,84355,"[Mark Duplass, Emily Blunt, Rosemarie DeWitt]",Lynn Shelton


In [26]:
total_data["overview"] = total_data["overview"].apply(lambda x: [str(x)])
total_data["genres"] = total_data["genres"].apply(lambda x: [str(genre) for genre in x])
total_data["keywords"] = total_data["keywords"].apply(lambda x: [str(keyword) for keyword in x])
total_data["cast"] = total_data["cast"].apply(lambda x: [str(actor) for actor in x])
total_data["crew"] = total_data["crew"].apply(lambda x: [str(crew_member) for crew_member in x])

total_data["tags"] = total_data["overview"] + total_data["genres"] + total_data["keywords"] + total_data["cast"] + total_data["crew"]
total_data["tags"] = total_data["tags"].apply(lambda x: ",".join(x).replace(",", " "))

total_data.drop(columns = ["genres", "keywords", "cast", "crew", "overview"], inplace = True)

total_data.iloc[5].tags

"['The seemingly invincible Spider-Man goes up against an all-new crop of villain – including the shape-shifting Sandman. While Spider-Man’s superpowers are altered by an alien organism  his alter ego  Peter Parker  deals with nemesis Eddie Brock and also gets caught up in a love triangle.'] Fantasy Action Adventure dual identity amnesia sandstorm love of one's life forgiveness spider wretch death of a friend egomania sand narcism hostility marvel comic sequel superhero revenge Tobey Maguire Kirsten Dunst James Franco S a m   R a i m i"

In [27]:
total_data.to_csv("clean_data.csv", index = False)

conn = sqlite3.connect("movies_database.db")

movies.to_sql("clean_movies_data", conn, if_exists = "replace", index = False)

4803

In [28]:
# KNN

vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(total_data["tags"])

model = NearestNeighbors(n_neighbors = 6, algorithm = "brute", metric = "cosine")
model.fit(tfidf_matrix)

def get_movie_recommendations(movie_title):
    movie_index = total_data[total_data["title"] == movie_title].index[0]
    distances, indices = model.kneighbors(tfidf_matrix[movie_index])
    similar_movies = [(total_data["title"][i], distances[0][j]) for j, i in enumerate(indices[0])]
    return similar_movies[1:]

input_movie = "Requiem for a Dream"
recommendations = get_movie_recommendations(input_movie)
print("Film recommendations '{}'".format(input_movie))
for movie, distance in recommendations:
    print("- Film: {}".format(movie))

Film recommendations 'Requiem for a Dream'
- Film: Trainspotting
- Film: Jesus' Son
- Film: Clean
- Film: Traffic
- Film: American Gangster
